In [2]:
import os
import pathlib

# # Clone the tensorflow models repository if it doesn't already exist
# if "models" in pathlib.Path.cwd().parts:
#   while "models" in pathlib.Path.cwd().parts:
#     os.chdir('..')
# elif not pathlib.Path('models').exists():
#   !git clone --depth 1 https://github.com/tensorflow/models

In [11]:
# Install the Object Detection API
# %%bash
#Change path
%cd /tf/models/research/

!protoc object_detection/protos/*.proto --python_out=.
%cp object_detection/packages/tf2/setup.py .
!python3 -m pip install .

/tf/models/research
Defaulting to user installation because normal site-packages is not writeable
Processing /tf/models/research
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 2.2 MB 34.3 MB/s            
     |████████████████████▊           | 322.4 MB 119.5 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |███████████████████████████     | 418.5 MB 89.9 MB/s eta 0:00:01 

     |███████████████████████████▎    | 424.0 MB 89.9 MB/s eta 0:00:01 

     |███████████████████████████▋    | 429.6 MB 89.9 MB/s eta 0:00:01 

...

     |███████████████████████████████▌| 490.1 MB 110.5 MB/s eta 0:00:01

     |███████████████████████████████▉| 495.1 MB 110.5 MB/s eta 0:00:01

     |████████████████████████████████| 497.6 MB 110.5 MB/s eta 0:00:01

     |████████████████████████████████| 497.6 MB 237 bytes/s           
     |████████████████████████████████| 4.9 MB 48.4 MB/s            
     |████████████████████████████████| 90 kB 22.3 MB/s            


     |████████████████████████████████| 1.4 MB 100.2 MB/s            
     |████████████████████████████████| 5.8 MB 60.3 MB/s            
     |████████████████████████████████| 462 kB 101.4 MB/s            
  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1684296 sha256=0ba05a034642c9ebc50f79ba809d6bd59ac0519397bf4548f28508ce810bab82
  Stored in directory: /tmp/pip-ephem-wheel-cache-ajyg8pws/wheels/89/ce/05/b0687c7bd22751a5e5a780a546434b4463698cb899345317b9
Successfully built object-detection
  Attempting uninstall: tf-models-official
    Found existing installation: tf-models-official 2.4.0
    Uninstalling tf-models-official-2.4.0:
      Successfully uninstalled tf-models-official-2.4.0
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1
ERROR: pip's dependency resolver does not currently take into account all

In [4]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
#from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

In [14]:
#run model builder test
!python3 /tf/models/research/object_detection/builders/model_builder_tf2_test.py

2022-03-01 15:47:02.056407: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-01 15:47:02.063321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-01 15:47:02.063723: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.8.5: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-03-01 15:47:02.071542: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critic

INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_faster_rcnn_models_from_config_mask_rcnn_without_matmul): 0.1s
I0301 15:47:06.441537 140662472578880 test_util.py:2373] time(__main__.ModelBuilderTF2Test.test_create_faster_rcnn_models_from_config_mask_rcnn_without_matmul): 0.1s
[       OK ] ModelBuilderTF2Test.test_create_faster_rcnn_models_from_config_mask_rcnn_without_matmul
[ RUN      ] ModelBuilderTF2Test.test_create_rfcn_model_from_config
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_rfcn_model_from_config): 0.1s
I0301 15:47:06.538719 140662472578880 test_util.py:2373] time(__main__.ModelBuilderTF2Test.test_create_rfcn_model_from_config): 0.1s
[       OK ] ModelBuilderTF2Test.test_create_rfcn_model_from_config
[ RUN      ] ModelBuilderTF2Test.test_create_ssd_fpn_model_from_config
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_ssd_fpn_model_from_config): 0.21s
I0301 15:47:06.744534 140662472578880 test_util.py:2373] time(__main__.ModelBuil

I0301 15:47:10.374283 140662472578880 ssd_efficientnet_bifpn_feature_extractor.py:142] EfficientDet EfficientNet backbone version: efficientnet-b2
I0301 15:47:10.374476 140662472578880 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet BiFPN num filters: 112
I0301 15:47:10.374542 140662472578880 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num iterations: 5
I0301 15:47:10.376373 140662472578880 efficientnet_model.py:147] round_filter input=32 output=32
I0301 15:47:10.392817 140662472578880 efficientnet_model.py:147] round_filter input=32 output=32
I0301 15:47:10.392995 140662472578880 efficientnet_model.py:147] round_filter input=16 output=16
I0301 15:47:10.523248 140662472578880 efficientnet_model.py:147] round_filter input=16 output=16
I0301 15:47:10.523430 140662472578880 efficientnet_model.py:147] round_filter input=24 output=24
I0301 15:47:10.765307 140662472578880 efficientnet_model.py:147] round_filter input=24 output=24
I0301 15:47:10.765495 14

I0301 15:47:14.273991 140662472578880 efficientnet_model.py:147] round_filter input=16 output=24
I0301 15:47:14.274165 140662472578880 efficientnet_model.py:147] round_filter input=24 output=32
I0301 15:47:14.549507 140662472578880 efficientnet_model.py:147] round_filter input=24 output=32
I0301 15:47:14.549674 140662472578880 efficientnet_model.py:147] round_filter input=40 output=56
I0301 15:47:15.080381 140662472578880 efficientnet_model.py:147] round_filter input=40 output=56
I0301 15:47:15.080542 140662472578880 efficientnet_model.py:147] round_filter input=80 output=112
I0301 15:47:15.579139 140662472578880 efficientnet_model.py:147] round_filter input=80 output=112
I0301 15:47:15.579335 140662472578880 efficientnet_model.py:147] round_filter input=112 output=160
I0301 15:47:16.039853 140662472578880 efficientnet_model.py:147] round_filter input=112 output=160
I0301 15:47:16.040085 140662472578880 efficientnet_model.py:147] round_filter input=192 output=272
I0301 15:47:16.789250 

I0301 15:47:23.063550 140662472578880 efficientnet_model.py:147] round_filter input=112 output=200
I0301 15:47:23.063712 140662472578880 efficientnet_model.py:147] round_filter input=192 output=344
I0301 15:47:23.866237 140662472578880 efficientnet_model.py:147] round_filter input=192 output=344
I0301 15:47:23.866404 140662472578880 efficientnet_model.py:147] round_filter input=320 output=576
I0301 15:47:24.077905 140662472578880 efficientnet_model.py:147] round_filter input=1280 output=2304
I0301 15:47:24.105126 140662472578880 efficientnet_model.py:457] Building model efficientnet with params ModelConfig(width_coefficient=1.8, depth_coefficient=2.6, resolution=528, dropout_rate=0.5, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_rati

In [15]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [16]:
#Change
train_record_fname = "/tf/carpartdetectiondataset/train/cars.tfrecord"
validation_record_fname = "/tf/carpartdetectiondataset/train/cars.tfrecord"
test_record_fname = "/tf/carpartdetectiondataset/test/cars.tfrecord"


In [17]:
label_map_pbtxt_fname = "/tf/carpartdetectiondataset/train/cars_label_map.pbtxt"

In [18]:
##change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 4
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 8
    }
}

In [19]:
chosen_model = 'efficientdet-d1'
classes = 18
num_steps = 875
num_eval_steps = 100 #Perform evaluation after so many steps

#numofsteps/numoftrainingdata * batch size
#Currently we have 350, 

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [20]:
#download pretrained weights

#Change
%mkdir /tf/models/research/deploy/
%cd /tf/models/research/deploy/
#
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

/tf/models/research/deploy
--2022-03-01 15:52:46--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.186.48, 2a00:1450:4001:827::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.186.48|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51839363 (49M) [application/x-tar]
Saving to: ‘efficientdet_d1_coco17_tpu-32.tar.gz’

100%[======================================>] 51,839,363   118MB/s   in 0.4s   

2022-03-01 15:52:47 (118 MB/s) - ‘efficientdet_d1_coco17_tpu-32.tar.gz’ saved [51839363/51839363]



In [21]:
#Change
%cd /tf/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/tf/models/research/deploy
--2022-03-01 15:53:01--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d1_640x640_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4630 (4.5K) [text/plain]
Saving to: ‘ssd_efficientdet_d1_640x640_coco17_tpu-8.config’

100%[======================================>] 4,630       --.-K/s   in 0s      

2022-03-01 15:53:02 (69.4 MB/s) - ‘ssd_efficientdet_d1_640x640_coco17_tpu-8.config’ saved [4630/4630]



In [22]:
#prepare
pipeline_fname = '/tf/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/tf/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [23]:
import re

#Change#
%cd /tf/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()

/tf/models/research/deploy
writing custom configuration file


In [24]:
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

In [33]:
#May not need this
#!pip install opencv
#!pip install -U --pre tensorflow_gpu=="2.2.0"

In [28]:
#Change
pipeline_file = '/tf/models/research/deploy/pipeline_file.config'
model_dir = '/tf/training/'

In [34]:
#Change
!python3 /tf/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

Traceback (most recent call last):
  File "/tf/models/research/object_detection/model_main_tf2.py", line 32, in <module>
    from object_detection import model_lib_v2
  File "/home/ec2-user/.local/lib/python3.8/site-packages/object_detection/model_lib_v2.py", line 30, in <module>
    from object_detection import inputs
  File "/home/ec2-user/.local/lib/python3.8/site-packages/object_detection/inputs.py", line 26, in <module>
    from object_detection.builders import model_builder
  File "/home/ec2-user/.local/lib/python3.8/site-packages/object_detection/builders/model_builder.py", line 37, in <module>
    from object_detection.meta_architectures import deepmac_meta_arch
  File "/home/ec2-user/.local/lib/python3.8/site-packages/object_detection/meta_architectures/deepmac_meta_arch.py", line 28, in <module>
    import tensorflow_io as tfio  # pylint:disable=g-import-not-at-top
  File "/home/ec2-user/.local/lib/python3.8/site-packages/tensorflow_io/__init__.py", line 17, in <module>
    f

In [ ]:
#Change
%load_ext tensorboard
%tensorboard --logdir '/content/training/train'

In [ ]:
EXPORTING

In [ ]:
#Change
#see where our model saved weights
%ls '/content/training/'

In [ ]:
#run conversion script
import re
import numpy as np

#Chnage
output_directory = '/content/fine_tuned_model'

#place the model weights you would like to export here
#Chang
last_model_path = '/content/training/'
print(last_model_path)
!python /content/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

In [6]:
!pip install tensorflow.contrib
from tensorflow.contrib import lite
converter = lite.TFLiteConverter.from_keras_model_file( 'isComplete.h5')
tfmodel = converter.convert()
open ("model.tflite" , "wb") .write(tfmodel)

ERROR: Could not find a version that satisfies the requirement tensorflow.contrib (from versions: none)
ERROR: No matching distribution found for tensorflow.contrib


ModuleNotFoundError: No module named 'tensorflow.contrib'